In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [65]:
def moments(row):
    idade = row['idade']
    renda_mensal = row['renda_mensal']
    investimento_total = row['investimento_total']
    
    if 0 <= idade < 16:
        return 'Momento 1'
    elif 16 <= idade < 26:
        return 'Momento 2'
    elif 26 <= idade < 60:
        if renda_mensal >= 5e3:
            if investimento_total < 2e5:
                return 'Momento 6'
            else:
                return 'Momento 7'
        elif (idade < 36) and (renda_mensal < 5e3):
            return 'Momento 3'
        elif (idade < 45) and (renda_mensal < 5e3):
            return 'Momento 4'
        elif renda_mensal < 5e3:
            return 'Momento 5'
    elif idade >= 60:
        if renda_mensal < 5e3:
            return 'Momento 8'
        else:
            return 'Momento 9'

In [76]:
df = pd.read_csv(
    'ldw-base_relacionamento_atual.csv',
    delimiter=';',
    dtype={'cod_carteira': str}
    )\
    .astype(
        {
            'cod_central': str,
            'cod_coop': str,
            'num_conta_principal': str,
            'cod_ua': str,
            'num_cpf_cnpj': str,
            'cod_cnae': str,
            'ano_mes': str,
            'flg_cheque_especial': str,
            'num_ano_mes': str,
            'flg_associado': str,
            'digital_acessou_30d': int,
            'mobi_acessou_30d': int
        }
    )\
    .assign(
        cad_pix=lambda x: x.cad_pix.map({'S': 1, 'N': 0}),
        assoc_desde=lambda x: pd.to_datetime(x.assoc_desde),
        tempo_assoc=lambda x: (pd.Timestamp('2024-11-01') - x.assoc_desde).dt.days,
        ultimo_contato=lambda x: pd.to_datetime(x.ultimo_contato),
        nivel_risco=lambda x: x.nivel_risco.fillna('DEFAULT'),
        score_principalidade=lambda x: x.score_principalidade.fillna(0),
        faixa_principalidade=lambda x: x.faixa_principalidade.replace('MISSING', np.nan).fillna('BAIXA'),
        publico_estrategico=lambda x: x.publico_estrategico.fillna('cooperativa'),
        investimento_total=lambda x: x.sld_lci + x.sld_lca + x.sld_fundos + x.sld_dep_a_prazo,
        qt_investimento_media=lambda x: x.loc[:, [
            'qt_fundos', 'qt_lci', 'qt_lca',
            'qt_dep_a_prazo']].mean(axis=1),
        alavancagem=lambda x: x.sld_cred_scr_2m / x.renda_mensal,
        flg_sexo=lambda x: [
            'PJ' if x.segmento.loc[i] == "PJ"
            else x.flg_sexo.loc[i]
            for i in range(x.shape[0])
        ],
        momentos=lambda x: x.apply(moments, axis=1)
    )\
    .query('(status_associado == "ATIVO") and (renda_mensal > 0)')\
    .drop(
        columns=[
            'sld_cred_scr_180_2m', 'sld_cred_sicredi_sas_2m',
            'sld_cred_sicredi_sas_180_2m', 'mc_seguro_agricola',
            'vlr_prej_coobrigacoes', 'vlr_prej_outros', 'sld_cred_moeda',
            'sld_seg_elementares', 'sld_seg_agricola', 'fat_ano',
            'mc_seg_elementares', 'mc_domicilio', 'mc_cred_moeda',
            'qt_cred_moeda', 'qt_seg_elementares', 'qt_seg_agricola'
        ]
    )\
    .dropna(subset=['idade'])\
    .reset_index(drop=True)

#df = df\
#    .astype(
#        {
#            col: str
#            for col in df.loc[:,df.columns.str.startswith(
#                ('prod', 'flg', 'digital',
#                 'possui', 'ib', 'mobi', 'fone')
#            )].columns
#        }
#    )\
#    .assign(
#        qt_investimento_media=lambda x: x.loc[:, [
#            'qt_fundos', 'qt_lci', 'qt_lca',
#            'qt_dep_a_prazo']].mean(axis=1)
#    )
#
#df.loc[(df.idade < 15) & (df.renda_mensal > 1e2), 'renda_mensal'] = 0
#df.loc[
#    (df.idade >= 19) &
#    (df.renda_mensal <= 100) &
#    (~df.desc_cbo.isin(['Estudantes'])),
#    'renda_mensal'] = np.nan
#df.loc[df.renda_mensal.isna(), 'renda_mensal'] = df\
#    .groupby('desc_cbo')\
#    .renda_mensal\
#    .transform('median')

In [74]:
#knn = KNNImputer(n_neighbors=10)
#knn_cols = df.dropna(subset=['renda_mensal'])\
#    .select_dtypes(include=np.number).corr(method='spearman')\
#    .renda_mensal.abs().sort_values(ascending=False).head(50).index\
#    .tolist()
#df_impute_knn = knn.fit_transform(df[knn_cols])
#df_impute_knn = pd.DataFrame(
#    df_impute_knn,
#    columns=knn_cols
#    )
#df['renda_mensal'] = df_impute_knn.renda_mensal

In [81]:
df_cluster = df\
    .loc[:, [
        'idade', 'renda_mensal',
        'tempo_assoc', 'score_principalidade',
        'isa', 
        #'cad_pix', 'digital_acessou_30d',
        #'mobi_acessou_30d', 
        'sld_previdencia',
        'qt_investimento_media',
        'alavancagem'] +
        df.columns[df.columns.str.startswith('sld_seg')].tolist() +
        df.columns[df.columns.str.startswith('sld_cred')].tolist()
    ]#\
    #.astype(
    #    {
    #        'cad_pix': int,
    #        'digital_acessou_30d': int,
    #        'mobi_acessou_30d': int
    #    }
    #)
df_cluster_columns = df_cluster.columns.tolist()

In [82]:
class Inertia: 
    def __init__(self, data, cluster_columns=None, set_groups=20):
        self.set_groups = set_groups + 1
        self.data = data
        self.data_cluster = data.loc[:, cluster_columns]
        self.cluster_columns = cluster_columns

    def find_groups(self, n_clusters):
        return KMeans(
            n_clusters=n_clusters,
            init='k-means++',
            random_state=42
            ).fit(self.data_cluster)\
            .labels_
    
    def calc_inertia(self):
        return [
            KMeans(
                n_clusters=i,
                init='k-means++',
                random_state=42
            ).fit(self.data_cluster)\
            .labels_
            for i in range(1, self.set_groups)
        ]

    def plot_inertia(self):
        plt.figure(figsize=(10, 8))
        plt.plot(
            range(1, self.set_groups),
            self.calc_inertia(),
            marker='o', 
            linestyle='--'
        )

In [83]:
#category_sizes = np.unique(df.porte_padrao, return_counts=True)
category_sizes = np.unique(df.momentos, return_counts=True)

In [84]:
kmeans = KMeans(
    n_clusters=9,
    init=df\
        .groupby('momentos')[df_cluster_columns]\
        .mean().to_numpy(),
        n_init=1,
        random_state=42,
    ).fit(df_cluster)
df = df\
    .assign(
        Grupos2=kmeans.labels_
    )

In [29]:
#for seed in range(30):
#    kmeans = KMeans(
#        n_clusters=9,
#        init=df\
#            .groupby('momentos')[df_cluster_columns]\
#            .mean().to_numpy(),
#            #.astype({
#            #    'digital_acessou_30d': int,
#            #    'mobi_acessou_30d': int
#            #})\
#            n_init=1,
#            random_state=seed,
#        ).fit(df_cluster)
#    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
#    print(f"Number of elements assigned to each cluster: {cluster_sizes}")
#print()
#print(
#    "True number of documents in each category according to the class labels: "
#    f"{category_sizes}"
#)

In [145]:
#teste = KMeans(
#    n_clusters=6,
#    init='k-means++',
#    random_state=42
#    ).fit(df_cluster)

In [ ]:
#df = df.assign(
#    Grupos=Inertia(df_cluster, df_cluster_columns).find_groups(6)
#    )

#df = df.assign(
#    Grupos=kmeans.labels_
#    )

In [147]:
stack_df = df.loc[:, df.columns[df.columns.str.startswith('prod_')].tolist() + ['Grupos']]\
    .set_index('Grupos')\
    .stack()\
    .reset_index(name='contem')\
    .astype({'contem': int})\
    .rename(columns={'level_1': 'produto'})\
    .groupby(['Grupos', 'produto'])\
    .contem\
    .sum()\
    .reset_index(name='total_produto_grupo')

In [12]:
#wcss = []
#for i in range(1, 21):
#    kmeans_pca = KMeans(
#        n_clusters=i,
#        init='k-means++',
#        random_state=42
#    )
#    kmeans_pca.fit(df_cluster)
#    wcss.append(kmeans_pca.inertia_)

In [13]:
#plt.figure(figsize=(10, 8))
#plt.plot(range(1, 21), wcss, marker='o', linestyle='--')

In [14]:
kmeans_pca = KMeans(n_clusters=6, init='k-means++', random_state=42)
kmeans_pca.fit(df_cluster)

KMeans(n_clusters=6, random_state=42)

In [15]:
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(scores_pca)], axis=1)
df.columns.values[-3:] = ['PC1', 'PC2', 'PC3']
df['Grupos'] = kmeans_pca.labels_.astype(str)
df = df.sort_values('Grupos')\
    .assign(
        Grupos=lambda x: x.Grupos.map(
            {
                '0': 'cluster 1',
                '1': 'cluster 2',
                '2': 'cluster 3',
                '3': 'cluster 4',
                '4': 'cluster 5',
                '5': 'cluster 6'
            }
        )
    )

In [20]:
#for grupo in df.Grupos.unique()[0]:
#    wcss = []
#    for i in range(1, 21):
#        kmeans = KMeans(
#            n_clusters=i,
#            init='k-means++',
#            random_state=42)
#        kmeans.fit(
#            df\
#                .query('Grupos == @grupo')\
#                .loc[:, df_cluster.columns.tolist()]
#        )
#        wcss.append(kmeans.inertia_)

In [ ]:
#qtd_prods = df_segm_pca_kmeans\
#    .loc[:, 
#         ['Grupos'] +
#         df_segm_pca_kmeans.columns[
#             df_segm_pca_kmeans.columns.str.startswith('prod_')
#            ].tolist()
#         ]\
#    .set_index('Grupos')\
#    .stack()\
#    .reset_index(name='possui')\
#    .rename(columns={'level_1': 'produto'})\
#    .astype({'possui': int})

In [86]:
df.to_csv('relacionamento_clusters1.csv', index=False)

In [ ]:
#df_segm_pca_kmeans.to_csv('relacionamento_clusters.csv', index=False)